<h2>1. kobart_finetuned 저장하고, 2. 아래 빨간색 글자대로 경로만 바꿔주고, 3. 위에서 아래로 순서대로 실행하시면 됩니다!</h2>

<h2>필요한 Library import하기</h2>

In [87]:
import os
import re
import argparse
from pathlib import Path
import pandas as pd
from tqdm import tqdm
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, AutoConfig

<h2>모델 호출 함수 정의</h2>

In [89]:
def load_tokenizer_model(local_dir: str):
    p = Path(local_dir)
    if not p.exists():
        raise FileNotFoundError(
            f"[ERROR] 로컬 모델 폴더를 찾을 수 없습니다: {local_dir}\n"
            "확인해주세요. (model.safetensors / tokenizer 파일들이 있어야 함)"
        )

    # config.json 수정: num_labels 같은 classification 필드 제거
    cfg_path = p / "config.json"
    try:
        if cfg_path.exists():
            import json
            with open(cfg_path, "r", encoding="utf-8") as f:
                cfg = json.load(f)
            for k in ["id2label", "label2id", "num_labels", "problem_type"]:
                cfg.pop(k, None)
            with open(cfg_path, "w", encoding="utf-8") as f:
                json.dump(cfg, f, ensure_ascii=False, indent=2)
    except Exception:
        pass

    cfg = AutoConfig.from_pretrained(local_dir, local_files_only=True)
    tokenizer = AutoTokenizer.from_pretrained(local_dir, use_fast=True, local_files_only=True)
    model = AutoModelForSeq2SeqLM.from_pretrained(local_dir, config=cfg, local_files_only=True)

    return tokenizer, model

<h2 style="color:red">모델 주소입력. MODEL_DIR의 경로를 꼭 모델을 저장한 경로에 맞춰서 바꿔주세요!!!!!!!</h2>

In [91]:
MODEL_DIR = r"C:\Users\sukoo\Documents\Kobart_finetuned\kobart_finetuned"
GLOBAL_TOKENIZER, GLOBAL_MODEL = load_tokenizer_model(MODEL_DIR)

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
GLOBAL_MODEL.to(DEVICE).eval()

BartForConditionalGeneration(
  (model): BartModel(
    (shared): BartScaledWordEmbedding(30000, 768, padding_idx=3)
    (encoder): BartEncoder(
      (embed_tokens): BartScaledWordEmbedding(30000, 768, padding_idx=3)
      (embed_positions): BartLearnedPositionalEmbedding(1028, 768)
      (layers): ModuleList(
        (0-5): 6 x BartEncoderLayer(
          (self_attn): BartAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (final_layer_n

<h2>요약 호출 함수 정의</h2>

In [93]:
def summarize_text(tokenizer, model, device, text: str, min_new_tokens: int, max_new_tokens: int) -> str:
    if not text or str(text).strip() == "":
        return ""
    inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=1024)
    inputs = {k: v.to(device) for k, v in inputs.items()}
    with torch.no_grad():
        out = model.generate(
            **inputs,
            do_sample=False,
            num_beams=5,
            min_new_tokens=min_new_tokens,
            max_new_tokens=max_new_tokens,
            no_repeat_ngram_size=3,
            repetition_penalty=1.2,
            pad_token_id=tokenizer.pad_token_id,
            eos_token_id=tokenizer.eos_token_id,
            early_stopping=True,
        )
    decoded = tokenizer.decode(out[0], skip_special_tokens=True).strip()
    cleaned = tidy_korean_summary(decoded)
    cleaned = drop_incomplete_sentences(cleaned)
    return cleaned

<h2>후처리 함수 정의</h2>

In [95]:
def tidy_korean_summary(text: str) -> str:
    t = re.sub(r"\s+", " ", (text or "")).strip()
    if t and t[-1] not in ".!?…。\"'”’":
        t += "."
    return t

def drop_incomplete_sentences(text: str) -> str:
    if not text or str(text).strip() == "":
        return ""
    sentences = re.split(r'(?<=[\.\?\!。\!?])\s*', text)
    sentences = [s.strip() for s in sentences if s and s.strip()]
    if len(sentences) == 0:
        return ""
    if len(sentences) == 1:
        last = sentences[0]
        if len(last.split()) < 3 or re.search(r'(며$|고$|하며$|으로$|에게$|서$|로$|에$|에 대해$|한다$|된다$)', last):
            return ""
        return last
    last = sentences[-1]
    if (len(last.split()) < 4) or (not re.search(r'[\.\?\!。\!?:;]$', last)) or re.search(r'(며$|고$|하며$|으로$|에게$|서$|로$|에$|에 대해$|한다$|된다$|것이다$)', last):
        sentences = sentences[:-1]
    cleaned = " ".join(sentences).strip()
    return cleaned

<h2>CVS 자동 읽기 함수 정의</h2>

In [97]:
def _read_table_auto(path: str, encoding: str = "utf-8"):
    ext = os.path.splitext(path)[1].lower()
    if ext in [".xlsx", ".xls"]:
        return pd.read_excel(path, engine="openpyxl")
    elif ext in [".tsv", ".tab"]:
        return pd.read_csv(path, sep="\t", encoding=encoding)
    else:
        return pd.read_csv(path, encoding=encoding)

<h2>csv 요약함수 정의</h2>

In [101]:
def summarize_csv(
    input_path: str,
    output_path: str,
    text_col: str = "newsContent",
    summary_col: str = "summary",
    min_tokens: int = 40,
    max_tokens: int = 150,
    encoding: str = "utf-8",
    device: str | None = None,
):
    df = _read_table_auto(input_path, encoding=encoding)

    print(f"[DEVICE] Using device: {DEVICE}")

    results = []
    for text in tqdm(df[text_col].fillna("").astype(str), desc="Summarizing", ncols=120):
        results.append(
            summarize_text(GLOBAL_TOKENIZER, GLOBAL_MODEL, DEVICE, text, min_tokens, max_tokens)
        )


    df_out = df.copy()
    df_out[summary_col] = results
    df_out.to_csv(output_path, index=False, encoding="utf-8-sig")

    print(f"[DONE] 요약 완료 → {output_path}")

<h2>단일 문자열 요약 함수 정의</h2>

In [115]:
def summarize_article(title: str, text: str) -> dict:
    """
    기사 제목(title)과 본문(text)을 입력받아
    text만 요약하고 {title, summary} 형태로 반환합니다.
    """
    summary = summarize_text(
        GLOBAL_TOKENIZER,
        GLOBAL_MODEL,
        DEVICE,
        text,
        min_new_tokens=40,
        max_new_tokens=150
    )

    return {
        "title": title,
        "summary": summary
    }


<h2>csv파일 요약 테스트. 요약하고 싶은 csv경로를 input_path에 지정하고, 결과물을 저장할 경로를 output_path에 넣으면 됩니다.</h2>

In [111]:
summarize_csv(
    input_path=r"C:\Users\sukoo\Documents\test.csv",
    output_path=r"C:\Users\sukoo\Documents\test_result.csv"
)

[DEVICE] Using device: cpu


Summarizing: 100%|████████████████████████████████████████████████████████████████████████| 4/4 [00:12<00:00,  3.04s/it]

[DONE] 요약 완료 → C:\Users\sukoo\Documents\test_result.csv


<h2>단일요약문 테스트. 기사의 title과 text를 summarize_article(title,text)로 넣어주시면 title과 summary가 딕셔너리로 저장됩니다!</h2>

In [123]:
title="""미국이 팩트시트 담으려 한 ‘중국 견제’ 문구…곳곳에 치열한 밀당 흔적"""

text="""4일 발표된 한미 정상회담 결과를 담은 공동 팩트시트에는 ‘중국’이란 단어는 한번도 등장하지 않았다. 하지만 미국의 중국 견제와 관련한 한·미 양국의 치열한 협상 결과가 반영됐다는 게 중론이다. 도널드 트럼프 미국 행정부는 ‘중국 견제’라는 전략적 목표를 세우면서도 미국의 부담은 줄이고 동맹들이 비용과 역할을 늘리는 ‘동맹 현대화’를 압박해 왔는데, 이번 한미 팩트시트에도 그 내용이 곳곳에 담겼다.

‘전략적 유연성’ 확대 요구, 적정선에서 방어

먼저 눈에 띄는 대목은 ‘역내 위협에 대한 재래식 억제 태세의 강화’다. “양국은 북한을 포함해 동맹에 대한 모든 역내의 위협에 대한 미국의 재래식 억제 태세를 강화할 것”이라는 표현에 담긴 ‘역내의 위협’은 사실상 중국을 염두에 둔 것으로 볼 수 있다. 한국은 이를 위해 “미국의 지원 아래 대북 연합 재래식 방위를 주도”하며, “필수적인 군사적 역량 강화 노력을 가속”하기로 했다. 이를 위해 한국은 △국내총생산(GDP)의 3.5%까지 국방비 증액 △2030년까지 미국산 군사장비 구매에 250억 달러를 지출 △주한미군에 대한 330억달러 규모의 종합적 지원 방침을 공유했다. 주한미군의 지속 주둔과 확장억제를 유지하는 대신 한국이 미국의 인도·태평양 전략에서 역할과 비용을 훨씬 더 많이 분담하기로 하는 ‘거래형 동맹’의 현실이다.

반면 ‘동맹 현대화’와 관련해 한미 양측이 “2006년 이래의 관련 양해를 확인한다”고 명시한 것은 주한미군의 ‘전략적 유연성’을 대폭 확대하려는 미국의 요구를 한국이 적절한 선에서 방어한 것으로 평가된다. 2006년 발표된 한미 공동성명에는 “전략적 유연성의 이행에 있어서, 미국은 한국이 한국민의 의지와 관계없이 동북아 지역분쟁에 개입되는 일은 없을 것이라는 한국의 입장을 존중한다”는 내용이 담겼다. 미국은 애초 이 조항을 삭제하고 주한미군이 제한 없이 대만 사태 등에 개입할 수 있는 길을 열려고 했지만, ‘2006년 수준’을 유지하려는 한국의 의도가 관철됐다.

반면 ‘한·미·일 3국 협력’이 재차 강조되고, 대만해협과 남중국해, 신장위구르 등 중국이 핵심이익으로 주장하는 지역과 관련된 내용이 담긴 것은 미국의 중국 견제 전략과 관련해 주목할 부분이다.

우선 팩트시트에는 ‘한·미·일 3자 협력을 강화하기로 했다’는 내용이 명시됐다. 이와 더불어 중국이 가장 민감하게 반응하는 대만 문제와 관련해서는 “대만해협에서의 평화와 안정 유지의 중요성”을 강조하고, “일방적 현상 변경에 반대”한다고 명시했다. 2023년 8월 한·미·일 캠프데이비드 선언에 담긴 ‘대만해협에서 힘에 의한 현상변경에 반대’한다는 내용과 거의 동일하다."""

result=summarize_article(title,text)

title=result["title"]

summary=result["summary"]

print("title: ",title,"summary: ",summary)

title:  미국이 팩트시트 담으려 한 ‘중국 견제’ 문구…곳곳에 치열한 밀당 흔적 summary:  도널드 트럼프 미국 행정부는 ‘중국 견제’라는 전략적 목표를 세우면서도 미국의 부담은 줄이고 동맹들이 비용과 역할을 늘리는 ‘동맹 현대화’를 압박해 왔는데, 이번 한미 팩트시트에도 그 내용이 곳곳에 담겼다.
